In [22]:
import random
import os
import mido
from mido import MidiFile,MidiTrack,Message

In [23]:
class MyMusic:           #定义音乐类

    
    '''第一部分 一些基本功能的静态方法'''

    @staticmethod
    def no_space(string):
        return ''.join(string.split(' '))
    @staticmethod
    def init0(string,l):                  #作用是把值为''的字符串初始化为'0'组成的对应长度字符串
        if string=='':
            return '0'*l
        else:
            return string
    @staticmethod
    def init4(string,l):                #作用是把值为''的字符串初始化为'4'组成的对应长度字符串
        if string=='':
            return '4'*l
        else:
            return string
    @staticmethod    
    def init1(string,l):                 #作用是把值为''的字符串初始化为'1'组成的对应长度字符串
        if string=='':
            return '1'*l
        else:
            return string
    @staticmethod
    def name_to_12(name):               #作用是将以'1234567'存储的音名转化为在12音中对应的数字，比如从1到1，从7到12
        
        if int(name)<=3:
            return (int(name)-1)*2+1
        else:
            return (int(name)-1)*2
    @staticmethod
    def time_to_num(time):               # 将用123456789abcdefg表示的时值转化为数
        if time.isdigit():
            return int(time)
        else:
            return ord(time)-ord('a')
       
    @staticmethod
    def name_to_pitch( name , mode='' , group='' , signature='C' , accidental='' ):            
                                        # 转换出12个半音的音高pitch序列
                                        # 完全按照mido库的写法，最低音为A0，记作21，之后每个半音会增加1
        pitch=[]
        length = len(name)
        mode = MyMusic.init0( mode , 7)
        group = MyMusic.init4( group , length)
        accidental = MyMusic.init0( accidental ,length)
        sig=MyMusic.name_to_12(ord(signature[0])-ord('C')+1)-1
        
        if ord(signature[0])<ord('C'):
            sig+=12
        
        if len(signature)>1:
            if signature == '+':
                sig+=1
            elif signature == '-':
                sig-=1
            else:
                print('signature error')
        for i in range(length):
            if name == '0':
                pitch.append[0]
                continue
                
            acci = None
            if accidental[i] == '=':
                acci = 0
            elif accidental[i]=='+' or mode[int(name[i])-1]=='+':
                acci = 1
            elif accidental[i] == '-' or mode[int(name[i]) - 1]== '-':
                acci = -1
            else:
                acci = 0
                
            flag = int(name[i])>5
            pitch.append( 11 + 12*int(group[i])  + MyMusic.name_to_12(name[i]) + acci +  sig )
        
        return pitch
    
    @staticmethod
    def interval_to_num(interval_name):               # 将 单音 / 音程 的字母序列转换成整数序列
        interval_num = []
        length = len(interval_name)
        for i in range(length):
            interval_num.append(ord(interval_name[i]) - 97)
        return interval_num
    
    '''
    根据音程的代号，求出音程的结构
    inteval_dict 是一个字典，索引是音程的整数代号，列表中的两个数字分别为根音和冠音与根音之间的半音数
    考虑到生成的随机音乐的 interval_num 可能超出 0-13 的范围，因此将 interval_num 对 14 取模，保证索引不越界
    
    '''
    
    @staticmethod
    def get_interval_arrangement(interval_num):
        interval_dict = {
            0 : [0, 0],
            1 : [0, 1],
            2 : [0, 2],
            3 : [0, 3],
            4 : [0, 4],
            5 : [0, 5],
            6 : [0, 6],
            7 : [0, 6],
            8 : [0, 7],
            9 : [0, 8],
            10 : [0, 9],
            11 : [0, 10],
            12 : [0, 11],
            13 : [0, 12]
        }
        interval = interval_dict[interval_num % 14]
        return interval
    
    @staticmethod
    def markov_generator(series):          # series是字符串或者list，内容是什么都行
                                            # 返回由这个可迭代序列给出的邻接矩阵 是一个词典
        
        trans={}                         # 转移矩阵是一个词典 keys是状态空间 values是一系列list 里边存对应键所有可能的后一个元素
                                            # 同一个元素出现几次就在list当中存几次 用于保证概率正确
        for i in series:                  #先遍历一遍 把series中出现过的加到trans的keys当中去  得到状态空间
            if i not in trans:
                trans[i]=[]
        last = None
        
        for i in series:                    #再遍历一遍，得到邻接矩阵
            if last != None:
                trans[last].append(i)
            last = i
        
        return trans
    
    @staticmethod
    def markov_series( trans , l = 50 , init = None ):    #给定邻接矩阵和长度，生成一个随机的list
                                                            #注意 要得到字符串还需要自行转化
        
        #默认第一个字符随机生成 否则可以通过init来指定
        
        series=[]
        if init == None:
            init= random.choice( list(trans.keys()) ) 
        series.append( init )
        for i in range( l-1 ):
            if not trans[init]:
                current= random.choice( list(trans.keys()) )
            else:
                current = random.choice( trans[init] )
            series.append( current )
            init = current
        
        return series

    @staticmethod
    def make_track( pitch , time_s='' , program=2 ):          # 由给定的音高和时值生成i个midi格式的音轨
                                                                # program表明音色，默认为2 钢琴
    
        time_s = MyMusic.init1(time_s,len(pitch))
        if len(pitch)!=len(time_s):
            print('length not match error')
            return
        track = MidiTrack()
        no=program
        meta_time = 60 * 60 * 5 /75
        meta_v=64
        gap_time = 0* meta_time
        track.append(Message('program_change', channel = 0 , program = no , time=0))
        track.append(Message('note_on', note=0 , velocity=meta_v, time= 32, channel=0))
        track.append(Message('note_off', note=0, velocity= meta_v, time=32, channel=0))
        
        for i in range(len(pitch)):
            track.append(Message('program_change', channel = 0 , program = no , time=0))
            track.append(Message('note_on', note =pitch[i]  , velocity=meta_v, time= round(gap_time), channel=0))
            track.append(Message('note_off', note=pitch[i] , velocity= meta_v, time= round(MyMusic.time_to_num(time_s[i])*meta_time*
                                                                                   0.5-gap_time), channel=0))
        
            
            
        return track
    
    '''第二部分 初始化函数
    
    最好不要修改这一部分 如果修改或者有bug请和我说明
    
    全都可以输入带任意空格的字符串，会自动去掉所有空格 所以可以用空格分小节来读谱 会比较好检查
    
    '''
    
    def __init__( self , name='' , signature = 'C' , mode = '' , meter = '44'   , group = '' , accidental = '' , time = '' , interval_name = ''
                 ,pitch='', interval_num = '', pitch1 = '' ):
                                                        # 初始化，所有不想要设定的值将会按照默认值
                                                        # 只有name音名字符串是必须填写的, 其余不填写将会被赋默认值
                                                        # 当你用简谱记谱的时候，请填写signature调号,即简谱的那个1=C 1=D之类  
                                                        # 注意，我们不考虑中途改变调性的歌曲，请不要选择这样的歌曲作为样本
                                                         # 当你用五线谱记谱的时候，不要填写signature，全局的升降号写在mode当中
        self._signature = MyMusic.no_space(signature)          # 调号  升降号用+ -表示  比如1=#A   那么就写'A+' 
        self._mode = MyMusic.no_space(mode)                    # 若为五线谱，可以给出调式，即在全局当中哪几个音名会被升调(+)/降调(-)，没有则可以不填
                                                                # 比如如果是 D和F升调 那么就应该输入 mode='0++0000'  请注意 这里一定是一个七个字符长的字符串
        self._meter= MyMusic.no_space(meter)                   # 节拍, 默认是44拍(四分音符为一拍，每小节四拍)，如果你的研究当中不使用可以不必理睬这个参数
        self._name=MyMusic.no_space(name)                      # 音名序列, 是一个由0到7的数字组成的字符串，1=C，以此类推(如果你设置了self.C为别的值那么会不同)
                                                                # 0表示休止符
        self._group=MyMusic.no_space(group)                    # 音组序列, 是一个数字组成的字符串，中央C是第四组的
                                                                # 休止符对应的音组、变音号填什么都无所谓
        self._accidental=MyMusic.no_space(accidental)          # 变音号序列， 是一个由0+-=四种字符组成的字符串，0代表无操作，+升号，-降号，=还原号
        self._time=MyMusic.no_space(time)                      # 时值序列，为了避免使用到小数，1代表1/4拍，多于9用a开始代表
                                                                #即：如果是44拍，那么16分音符为1,全音符为g。不考虑更小的时值。
        self._interval_name=MyMusic.no_space(interval_name)                # 和弦序列, 我不知道怎么记你会比较方便做，所以可能需要你自己规定
        
        if pitch =='':
            self._pitch = MyMusic.name_to_pitch(self._name,self._mode,self._group,self._signature,self.accidental)
        else:
            self._pitch=pitch                          #pitch是12音下的绝对音高数据，一般用于被生成的音乐，不会用于读谱
            
        if interval_num == '':
            self._interval_num = MyMusic.interval_to_num(interval_name)
        else:
            self._interval_num = interval_num
        
        self._pitch1 = []
        if pitch1 == '':
            for i in range(len(self._pitch)):
                interval = MyMusic.get_interval_arrangement(self._interval_num[i])
                self._pitch1.append(self._pitch[i] + interval[1])
        else:
            self._pitch1 = pitch1
            
        

    '''第三部分 用马尔可夫方式生成随机音乐的相关函数''' 
    '''
    这一部分已经给出的函数是根据音乐的name、signature、mode、group、accidental这五个参数（实际上 它们只是为了得到一个pitch序列）
    和音乐时值得到的随机音乐
    
    '''


    def create_music(self ,l=50):         #l用于给定生成乐段的音符数长短  已经考虑了音高和时值
        
        if self._time=='':
            return MyMusic(pitch = MyMusic.markov_series(MyMusic.markov_generator(self._pitch),l), 
                            interval_num = MyMusic.markov_series(MyMusic.markov_generator(self._interval_num), l))
        else:
            return MyMusic(pitch = MyMusic.markov_series(MyMusic.markov_generator(self._pitch),l),
                           interval_num = MyMusic.markov_series(MyMusic.markov_generator(self._interval_num), l),
                           time= ''.join(MyMusic.markov_series(MyMusic.markov_generator(self._time),l)))

        
    '''第四部分 播放本音乐'''

    def play(self,program=2):                  #用于播放音乐，program为音色 默认为2 钢琴
        mid = MidiFile()
        mid.tracks.append(MyMusic.make_track(self._pitch, self._time, program))
        mid.tracks.append(MyMusic.make_track(self._pitch1, self._time, program))
        mid.save('tmp.mid')
        os.system('tmp.mid')  
        return
     
    def save(self, filename='test.mid',program=2):            #保存音乐 可以指定文件名
        mid = MidiFile()
        mid.tracks.append(MyMusic.make_track(self._pitch, self._time,program))
        mid.tracks.append(MyMusic.make_track(self._pitch1, self._time, program))
        mid.save(filename)
        return
         
    
    
    '''第五部分 用于外部访问本类属性的函数'''
    @property
    def signature(self):
        return self._signature
    
    @property
    def mode(self):
        return self._mode
    
    @property
    def signature(self):
        return self._signature
    
    @property
    def meter(self):
        return self._meter
    
    @property
    def name(self):
        return self._name
    
    @property
    def accidental(self):
        return self._accidental
    
    @property
    def time(self):
        return self._time
    
    @property
    def interval(self):
        return self._interval
    
    @property
    def pitch(self):
        return self._pitch

测试：输入乐曲 喀秋莎音程版

In [24]:
'''
喀秋莎音程版
signature  = 'C'
name = '37 66 6654 53 51 75 7765 3'
mode = '0000000'
meter = '42'
group = '44 44 4444 44 45 44 4444 4'
accidental = '00 00 00++ +0 +0 0+ 000+ 0'
time = '62 62 2222 44 62 62 2222 8'
chord = 'fa  da dddd da da dd dddd f'

'''
a=MyMusic(name = '37 66 6654 53 51 75 7765 3',
          signature  = 'C',
          mode = '0000000',
          meter = '42',
          group = '44 44 4444 44 45 44 4444 4',
          accidental = '00 00 00++ +0 +0 0+ 000+ 0',
          time = '62 62 2222 44 62 62 2222 8',
          interval_name = 'fa  da dddd da da dd dddd f')

a.save('kaqiusha_interval.mid')


测试：根据 喀秋莎音程版 生成一段默认长度（l=50）的音乐

In [25]:

b=a.create_music()
b.save('kaqiusha_inteval_creation.mid')

测试：输入乐曲 克罗地亚狂想曲音程版

In [26]:
'''
克罗地亚狂想曲音程版
signature = 'C'
name = '231231564564  6755231231  23123515645614  6755231231  2315123151  231171231  2315123151  231171231   23151564  6755231231  23151564  6755231231 2315123151  231171231  2315123151  231171231'
mode = '00-00--'
meter = '44'
group = '666666555555  5555555555 66666665555565  5555555555  5555555555  555545555  5555555555  555545555  55555444  4444444444  55555444  4444444444  4444444444  444434444  4444444444  444434444'
accidental = '000000000000  ==00000000  00000000000000  ==00000000  0000000000  0000=0000  0000000000  0000=0000  00000000  ==00000000  00000000  ==00000000  0000000000  0000=0000  0000000000  0000=0000'
time = '112112112112  1122112114  11211111121111  1122112114  1141111411  112213114  1141111411  112213114  11411116  1122112114  11411116  1122112114  1141111411  112213114  1141111411  112213114'
chord = 'faafaafaafaa  faaifaafaa  faafaaafaafaaa  faaifaafaa  faaaafaaaa  faaiaifaa  faaaafaaaa  faaiaifaa  faaaafaa  faaifaafaa  faaaafaa  faaifaafaa  faaaafaaaa  faaiaafaa  faaaafaaaa  faaiaafaa'

'''
a = MyMusic(signature = 'C',
            name = '231231564564  6755231231  23123515645614  6755231231  2315123151  231171231  2315123151  231171231   23151564  6755231231  23151564  6755231231 2315123151  231171231  2315123151  231171231',
            mode = '00-00--',
            meter = '44',
            group = '666666555555  5555555555 66666665555565  5555555555  5555555555  555545555  5555555555  555545555  55555444  4444444444  55555444  4444444444  4444444444  444434444  4444444444  444434444',
            accidental = '000000000000  ==00000000  00000000000000  ==00000000  0000000000  0000=0000  0000000000  0000=0000  00000000  ==00000000  00000000  ==00000000  0000000000  0000=0000  0000000000  0000=0000',
            time = '112112112112  1122112114  11211111121111  1122112114  1141111411  112213114  1141111411  112213114  11411116  1122112114  11411116  1122112114  1141111411  112213114  1141111411  112213114',
            interval_name = 'faafaafaafaa  faaifaafaa  faafaaafaafaaa  faaifaafaa  faaaafaaaa  faaiaifaa  faaaafaaaa  faaiaifaa  faaaafaa  faaifaafaa  faaaafaa  faaifaafaa  faaaafaaaa  faaiaafaa  faaaafaaaa  faaiaafaa')

a.save('Croatian.mid')

测试：根据 克罗地亚狂想曲音程版 生成一段默认长度（l = 50）的乐曲

In [27]:
b=a.create_music()
b.save('Croatian_creation.mid')